In [1]:
import os, glob
import numpy as np
import tensorflow as tf
from PIL import Image

from sklearn.model_selection import train_test_split

from keras.layers import MaxPooling2D
from keras.layers import Conv2D
from keras.models import Model
from keras.layers import Activation, Dropout, Flatten, Dense, Input, BatchNormalization, Concatenate, GlobalAveragePooling2D
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

from tensorflow.keras.applications.resnet50 import ResNet50

from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array, load_img, array_to_img

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# seed 고정
seed = 42

In [4]:
# folder_list = os.listdir('./drive/MyDrive/faces/남자')
folder_list = os.listdir('./faces/남자')

In [7]:
# # 이미지 증강용
# def IDG(fname): #파일이름을 변수로 받는 ImageDataGenerator함수
#    ImageDG = ImageDataGenerator(
#        rescale = 1. / 255, # 이미지 value 정규화화
#        rotation_range = 90, # 무작위 회전의 각도 범위
#        brightness_range = [0.3,1.0], # 밝기 범위위
#        fill_mode='nearest' # 인풋 경계의 바깥공간 채우는 방식
#        )
   
#    img = tf.keras.preprocessing.image.load_img(fname) # 이미지파일로 변환
#    x = img_to_array(img)
#    x = x.reshape((1,) + x.shape)
#    i=1
#    save = fname.split('/')[0] + "/" + fname.split('/')[1] + "/" + fname.split('/')[2] + "/" + fname.split('/')[3]
#    # 생성된 파일 저장경로
#    for batch in ImageDG.flow(x, batch_size=1, save_to_dir = save, save_prefix='new'+fname.split('/')[3], save_format='png'):
#        i+=1
#        if i>=10: # 새로 생성되는 데이터 개수 정해주기
#            break

In [8]:
# # 이미지 증강용
# folder_list = os.listdir('./faces/남자')
# fname =  './faces/남자/'
# for f in folder_list:
#     fname =  './faces/남자/' + f +"/"
#     file_list = os.listdir(fname)
#     for i in file_list:
#         filename = fname + i
#         IDG(filename)

In [4]:
# 분류 대상 카테고리 선택하기 
faces_dir = './faces/남자'
man_categories = [
    '견우','서준','수호','시호','준우','태양','태호','혜성'
]
nb_classes = len(man_categories)
# 이미지 크기 지정 
image_w = 256
image_h = 256
pixels = image_w * image_h * 3
# 이미지 데이터 읽어 들이기 
X = []
Y = []
for idx, cat in enumerate(man_categories):
    # 레이블 지정 
    label = [0 for i in range(nb_classes)]
    label[idx] = 1
    # 이미지 
    image_dir = faces_dir + "/" + cat
    files = glob.glob(image_dir+"/*.png")
    for i, f in enumerate(files):
        img = Image.open(f)
        img = img.convert("RGB")
        img = img.resize((image_w, image_h))
        data = np.asarray(img)      # numpy 배열로 변환
        X.append(data)
        Y.append(label)
X = np.array(X)
Y = np.array(Y)

In [5]:
print(X.shape)
print(Y.shape)

(5068, 256, 256, 3)
(5068, 8)


In [6]:
# X_std = X/255.0

In [7]:
resnet = ResNet50(weights='imagenet', include_top=False)

In [8]:
for layer in resnet.layers:
    layer.trainable = False

In [9]:
inputs = Input(shape=(256,256,3))
res = resnet(inputs)
gap = GlobalAveragePooling2D()(res)
dense = Dense(128, activation='relu')(gap)
outputs = Dense(nb_classes, activation='softmax')(dense)

In [10]:
model = Model(inputs=inputs, outputs=outputs)

In [11]:
# 모델 확인
print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, None, None, 2048)  23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 128)               262272    
                                                                 
 dense_1 (Dense)             (None, 8)                 1032      
                                                                 
Total params: 23,851,016
Trainable params: 263,304
Non-trainable params: 23,587,712
___________________________________________

In [12]:
model.compile(loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

In [13]:
# 학습 전용 데이터와 테스트 전용 데이터 구분 
X_train, X_test, y_train, y_test = \
    train_test_split(X_std, Y, random_state=seed, test_size=0.2)

In [14]:
# 클래스 포함 확인
import pandas as pd
for i in range(len(pd.DataFrame(y_test).columns)):
    print(pd.DataFrame(y_test)[i].value_counts())

0    873
1    141
Name: 0, dtype: int64
0    880
1    134
Name: 1, dtype: int64
0    888
1    126
Name: 2, dtype: int64
0    901
1    113
Name: 3, dtype: int64
0    899
1    115
Name: 4, dtype: int64
0    899
1    115
Name: 5, dtype: int64
0    868
1    146
Name: 6, dtype: int64
0    890
1    124
Name: 7, dtype: int64


In [15]:
# checkpoint_filepath = "MFMbest.hdf5"

# early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=1)

# save_best = tf.keras.callbacks.ModelCheckpoint(
#     filepath=checkpoint_filepath, monitor='val_loss', verbose=1, save_best_only=True,
#     save_weights_only=True, mode='auto', save_freq='epoch', options=None)

In [16]:
X_train.shape

(4054, 256, 256, 3)

In [17]:
X_test.shape

(1014, 256, 256, 3)

In [18]:

hdf5_file = "MFMbest.hdf5"
if os.path.exists(hdf5_file):
    # 기존에 학습된 모델 불러오기기
    model.load_weights(hdf5_file)
else:
    # 학습한 모델이 없으면 파일로 저장
    model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))
    model.save_weights(hdf5_file)

Epoch 1/10
127/127 [==============================] - 360s 3s/step - loss: 1.6967 - accuracy: 0.3493 - val_loss: 1.4193 - val_accuracy: 0.4941
Epoch 2/10
127/127 [==============================] - 331s 3s/step - loss: 1.3475 - accuracy: 0.5010 - val_loss: 1.1674 - val_accuracy: 0.5848
Epoch 3/10
127/127 [==============================] - 338s 3s/step - loss: 1.1239 - accuracy: 0.6014 - val_loss: 1.0344 - val_accuracy: 0.6134
Epoch 4/10
127/127 [==============================] - 343s 3s/step - loss: 0.9664 - accuracy: 0.6591 - val_loss: 0.8991 - val_accuracy: 0.6677
Epoch 5/10
127/127 [==============================] - 350s 3s/step - loss: 0.8605 - accuracy: 0.6998 - val_loss: 0.8025 - val_accuracy: 0.7288
Epoch 6/10
127/127 [==============================] - 341s 3s/step - loss: 0.7574 - accuracy: 0.7474 - val_loss: 0.7346 - val_accuracy: 0.7367
Epoch 7/10
127/127 [==============================] - 344s 3s/step - loss: 0.7028 - accuracy: 0.7590 - val_loss: 0.6357 - val_accuracy: 0.7929